# <center><a href='https://notebook.basthon.fr/?from=https://raw.githubusercontent.com/lchalmain/mpsi-itc/main/tp10_elections_dico_corrige.ipynb'>TP 10 : Dépouillement d'élections - dictionnaires <img src=https://framagit.org/uploads/-/system/project/avatar/55763/basthon_shadow.png width=100></a></center>

<center><h1 style="color:red">Corrigé</h1></center>

# Dépouillement d'élections à l'aide de dictionnaire

On reprend les éléments du concours blanc, que l'on cherche à traiter à l'aide de dictionnaires.

Dans une classe de $n$ étudiants ($n\in \mathbb{N}^\star$), chaque étudiant est identifié par un entier compris entre $0$ et $n-1$.

Pour élire le(s) délégué(e)(s) de la classe chaque étudiant peut voter pour n'importe lequel de ses camarades, y compris lui-même, ou voter blanc.

Un vote est représenté par une liste `liste_votes` telle que :
* `liste_votes[i] == j` lorsque l'étudiant $i$ a voté pour l'étudiant $j$.
* `liste_votes[i] == None` lorsque l'étudiant $i$ a voté blanc.

On considère que chaque étudiant vote. La liste de votes contient donc autant d'éléments que le nombre d'étudiants.

Le scrutin est considéré comme **valide** lorsque le nombre de votes exprimés (i.e. non blancs) est strictement supérieur à la moitié du nombre de votants.

**<span style = "color:purple">Exercice :</span>** 

1. Écrire une fonction `ajoute` telle que `ajoute(cle, val, dico)` ajoute la clé `cle` associée à la valeur `val` si la clé n'est pas encore présente dans le dictionnaire `dico` et incrémente de `val` la valeur associée sinon.
1. Écrire une fonction `compte` telle que `compte(liste_votes)` renvoie un dictionnaire comprenant les différents éléments de `liste_votes` (les étudiants pour qui on a voté) comme clés et le nombre d'occurences de chacun comme valeur associée (le nombre de votes obtenus par l'étudiant en question). On comptera également le nombre de votes blancs.<br>
*Indication :* lorsqu'on parcours la liste des votes, si un élément n'est pas encore présent dans le dictionnaire, il faut l'ajouter, sinon il faut augmenter la valeur associée.
1. En déduire une fonction `estValide` telle que `estValide(liste_votes)` renvoie `True` lorsque le vote correspondant à `liste_votes` est valide et `False` sinon.
<hr>

In [1]:
#1
def ajoute(cle, val, dico):
    if not (cle in dico):  # cle n'est pas encore dans dico
        dico[cle] = val
    else:
        dico[cle] = dico[cle] + val

In [2]:
#2
def compte(liste_votes):
    dico = {None : 0}  # Nécessaire pour disposer du nb de votes blancs lorsqu'il n'y en a pas
    for vote in liste_votes:
        ajoute(vote, 1, dico)
    return dico

compte([1, 1, 2, None, 4, None, 1])

{None: 2, 1: 3, 2: 1, 4: 1}

In [3]:
#3
def estValide(liste_votes):
    nb_votes = len(liste_votes)
    exprimes = nb_votes - compte(liste_votes)[None]  # il y a nécessairement une clé None dans le dico
    return exprimes > nb_votes / 2

assert(estValide([1, 1, 2, None, 4, None, 1]))

assert(not (estValide([1, None, 2, None, 4, None])) )  #6/2 = 3, il faudrait au moins 4 votes exprimés

# Premier mode de scrutin

Chaque étudiant ne vote qu'une fois. L'étudiant élu est celui qui a reçu le plus de voix (si le vote est valide). S'il y en a plusieurs, ils sont tous élus.

**<span style = "color:purple">Exercice :</span>** 

1. Écrire une fonction `maximum` telle que `maximum(dico_votes)` renvoie le nombre maximal de votes obtenu par un étudiant dans le dictionnaire `dico_votes` (hors votes blancs).<br>
La fonction renverra `None` s'il n'y a que des votes blancs.

1. En déduire une fonction `listeMax` telle que `listeMax(dico_votes)` renvoie la liste des étudiants ayant obtenu le nombre maximal de votes dans le dictionnaire `dico_votes`. On supposera qu'il n'y a pas que des votes blancs.

1. Ecrire finalement une fonction `scrutin1` telle que `scrutin1(liste_votes)` renvoie la liste des étudiants élus lorsque le vote est valide et `"Vote non valide"` sinon.
<hr>

In [4]:
#1
def maximum(dico_votes):
    maxi = None
    for etudiant, nb_votes in dico_votes.items():
        if etudiant != None:
            if maxi == None:    # c'est le premier vote non blanc
                maxi = nb_votes
            elif nb_votes > maxi:
                maxi = nb_votes
    return maxi

assert(maximum({1: 3, 2: 1, None: 2, 4: 1}) == 3)

In [5]:
#2
def listeMax(dico_votes):    # on suppose qu'il n'y a pas que des votes blancs
    elus = []
    maxi = maximum(dico_votes)
    for etudiant, nb_votes in dico_votes.items():
        if etudiant != None and nb_votes == maxi:
            elus.append(etudiant)
    return elus
        
assert(listeMax({1: 3, 5: 3, None: 2, 4: 1}) == [1, 5])

In [6]:
#3
def scrutin1(liste_votes):
    if not estValide(liste_votes):
        return "Vote non valide"
    dico_votes = compte(liste_votes)
    return listeMax(dico_votes)

assert( scrutin1([1, None, 1, 2, 3]) == [1])
assert( scrutin1([1, None, 1, 2, 2]) == [1, 2])
assert(scrutin1([1, None, 2, None, 4, None]) == "Vote non valide")

# Second mode de scrutin

Chaque étudiant vote deux fois : une première fois pour son candidat favori (les résultats sont contenus dans une liste `L1`) et une seconde fois pour son deuxième candidat préféré (liste `L2`). Les deux étudiants ayant reçu le plus de voix sont élus. Si plusieurs candidats ont reçu le même nombre de voix, on sélectionnera le premier ayant reçu un vote en sa faveur, dans l'ordre de vote des étudiants.

Pour que le vote soit valide il faut que la liste `L1` soit valide au sens du premier mode de scrutin et qu'aucun étudiant n'ait effectué deux votes identiques, sauf éventuellement blancs.

**<span style = "color:purple">Exercice :</span>** 

1. Écrire une fonction `compte2` telle que `compte2(L1, L2)` renvoie un dictionnaire comprenant les différents éléments de `liste_votes` (les étudiants pour qui on a voté) comme clés et le nombre de votes obtenus par l'étudiant en question comme valeur associée (les votes de `L1` doivent compter double). La fonction devra renvoyer `"Vote non valide"` si le vote n'est pas valide.<br>
*Indication :* on pourra utiliser la fonction `ajoute` définie précédemment.
1. Écrire une fonction `couple2Max` telle que `couple2Max(dico_votes)` renvoie les deux étudiants ayant reçu le maximum de votes dans le dictionnaire passé en argument. Si deux étudiants ont reçu le même nombre de voix, c'est le premier étudiant présent dans le dictionnaire qui est sélectionné (cela correspond à la description du mode de scrutin).
1. Écrire finalement une fonction `scrutin2` telle que `scrutin2(L1, L2)` renvoie `"Vote non valide"` si le vote n'est pas valide, `"Un seul étudiant a remporté des voix"` s'il n'y a qu'un étudiant qui a reçu des voix et le couple d'étudiants élus sinon.
<hr>

In [7]:
#1
def compte2(L1, L2):  # on suppose que L1 et L2 sont de même taille
    nb = len(L1)
    blancs = 0
    dico = {}
    for i in range(nb):
        if L1[i] != None and L1[i] == L2[i]:  # l'étudiant i a voté deux fois pour le même étudiant
            return "Vote non valide"
        else:
            ajoute(L1[i], 2, dico)  # les votes de la liste L1 comptent double
            ajoute(L2[i], 1, dico)
        if L1[i] == None:    # l'étudiant i a effectué un vote blanc en premier
            blancs = blancs + 1
    exprimes = nb - blancs
    if exprimes <= nb / 2:  # L1 correspond à un vote non valide selon le premier mode de scrutin
        return "Vote non valide"
    return dico

assert(compte2([1, 0, 0], [0, 1, 0]) == "Vote non valide")
compte2([1, 2, None], [None, 1, None])

{1: 3, None: 4, 2: 2}

In [8]:
#2
def couple2Max(dico_votes):    
    maxi1, elu1 = 0, None
    maxi2, elu2 = 0, None
    for etudiant, nb_votes in dico_votes.items():
        if etudiant != None :
            if nb_votes > maxi1:
                maxi2, elu2 = maxi1, elu1
                maxi1, elu1 = nb_votes, etudiant
            elif nb_votes > maxi2:
                maxi2, elu2 = nb_votes, etudiant
    return elu1, elu2
        
assert(couple2Max({1: 3, 2: 4, 5: 3, None: 2, 4: 1}) == (2, 1))

In [9]:
#3
def scrutin2(L1, L2):
    dico_votes = compte2(L1, L2)
    if dico_votes == "Vote non valide":
        return "Vote non valide"
    else:
        elu1, elu2 = couple2Max(dico_votes)
        if elu2 == None:
            return "Un seul étudiant a remporté des voix"
        else:
            return elu1, elu2
    
assert(scrutin2([1, 2, None, 1], [None, 1, None, 0]) == (1, 2))
assert(scrutin2([1, 2, None, None], [None, 1, None, 0]) == "Vote non valide")
scrutin2([1, None, 1], [None, None, None])

'Un seul étudiant a remporté des voix'